In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import scipy.io.wavfile as wavfile
from io import *
import os.path

In [11]:
# From TFRecord Tutorial
# The following functions can be used to convert a value to a type compatible

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [12]:
def slice_signal(signal, window_size, stride=0.5):
    """ Return windows of the given signal by sweeping in stride fractions
        of window
    """
    assert signal.ndim == 1, signal.ndim
    n_samples = signal.shape[0]
    offset = int(window_size * stride)
    slices = []
    for beg_i in range(0, n_samples, offset):
        end_i = beg_i + offset
        slice_ = signal[beg_i:end_i]
        # pad the rest of the slice with 0
        if end_i - beg_i > n_samples - beg_i:
            slice_ = signal[beg_i:n_samples]
            test_l = np.zeros((offset-(n_samples-beg_i)))
            slice_ = np.concatenate((slice_,test_l),axis=None)
        if slice_.shape[0] == offset:
            slices.append(slice_)
    return np.array(slices, dtype=np.int32)

In [ ]:
def make_tf_example(sequence, label):
    example = tf.train.SequenceExample()
    #features=tf.train.Features(feature={'wav_raw': _int64_feature(audio_serial), 'label': _bytes_feature([lbl])}))

    return example

In [13]:
# Define the data
DATADIR = r".\Dataset"
LABELS = ["clean", "noisy"]

for lbl in LABELS:
    path = os.path.join(DATADIR, lbl)  # create path to sequences
    save_path = os.path.join(DATADIR,"records",lbl+"_save01.tfrecords")
    out_file = tf.io.TFRecordWriter(save_path) # instantiate the recorder
    for file in os.listdir(path):  # iterate over each image per dogs and cats
        #audio_serial = read_and_slice(os.path.join(path,file), 2 ** 14, stride=1)
        fm, wav_data = wavfile.read(os.path.join(path,file)) # read the wav file
        audio_serial = slice_signal(wav_data, 2 ** 14, stride=1) # transform it into a np.array
        if fm != 16000: # check it sampling
            raise ValueError('Sampling rate is expected to be 16kHz!')
    
    


TypeError: only size-1 arrays can be converted to Python scalars

In [ ]:
bc = BertClient()
list_vec = bc.encode(lst_str)
list_label = [0 for _ in lst_str]  # a dummy list of all-zero labels

# write to tfrecord
with tf.python_io.TFRecordWriter('tmp.tfrecord') as writer:
    def create_float_feature(values):
        return tf.train.Feature(float_list=tf.train.FloatList(value=values))

    def create_int_feature(values):
        return tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))

    for (vec, label) in zip(list_vec, list_label):
        features = {'features': create_float_feature(vec), 'labels': create_int_feature([label])}
        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())